## Setup

In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
print('Tensorflow version is', tf.__version__)

Tensorflow version is 1.12.0


## Load Dataset

In [10]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
print('Training data', len(train_data))
print('Testing data', len(test_data))

Training data 25000
Testing data 25000


## Data Exploration

In [3]:
print('The first training sample:', train_data[0])
print('The first training sample\'s label:', train_labels[0])

The first training sample: [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
The first training sample's label

In [4]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
  
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

## Padding

In [5]:
train_data_pd = keras.preprocessing.sequence.pad_sequences(
    train_data,
    value=word_index["<PAD>"],
    padding='post',
    maxlen=256)

test_data_pd = keras.preprocessing.sequence.pad_sequences(
    test_data,
    value=word_index["<PAD>"],
    padding='post',
    maxlen=256)

print('Shape of padded training set', train_data_pd.shape)
print('Shape of padded testing set', test_data_pd.shape)
print(train_data_pd[0])
print(decode_review(train_data_pd[0]))

Shape of padded training set (25000, 256)
Shape of padded testing set (25000, 256)
[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134

## Experimental Protocol

In [0]:
x_validation = train_data_pd[:10000]
x_train = train_data_pd[10000:]
x_test = test_data_pd

y_validation = train_labels[:10000]
y_train = train_labels[10000:]
y_test = test_labels

## Training

In [22]:
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 25))
model.add(keras.layers.CuDNNGRU(100))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])



history = model.fit(x_train,
                    y_train,
                    epochs=15,
                    batch_size=512,
                    validation_data=(x_validation, y_validation),
                    verbose=1)
results = model.evaluate(x_test, y_test)

print(results)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 15000 samples, validate on 10000 samples
Epoch 1/15
15000/15000 [==============================] - 5s 311us/step - loss: 0.6931 - acc: 0.5067 - val_loss: 0.6924 - val_acc: 0.5186
Epoch 2/15
15000/15000 [==============================] - 3s 185us/step - loss: 0.6881 - acc: 0.5473 - val_loss: 0.6739 - val_acc: 0.5529
Epoch 3/15
15000/15000 [==============================] - 3s 187us/step - loss: 0.7071 - acc: 0.5456 - val_loss: 0.6956 - val_acc: 0.5219
Epoch 4/15
15000/15000 [==============================] - 3s 185us/step - loss: 0.6733 - acc: 0.5568 - val_loss: 0.6818 - val_acc: 0.5334
Epoch 5/15
15000/15000 [==============================] - 3s 185us/step - loss: 0.6579 - acc: 0.5813 - val_loss: 0.6738 - val_acc: 0.5428
Epoch 6/15
15000/15000 [==============================] - 3s 186us/step - loss: 0.6412 - acc: 0.5993 - val_loss: 0.6666 - val_acc: 0.5489
Epoch 7/15
15000/15000 [==============================] - 3s 186us/step - loss: 0.6208 - acc: 0.6129 - val_loss: 0.6612 - 